In [9]:
from __future__ import (absolute_import, division, print_function,
                        unicode_literals)

import datetime
from backtrader import cerebro  # For datetime objects
import pandas as pd
import backtrader as bt
from pandas.core.frame import DataFrame
import tushare as ts
import logging


# Create a Stratey
class TestStrategy(bt.Strategy):
    
    params = (
        ('exitbars', 5),
        ('maperiod', 5)
    )

    def log(self, txt, dt=None):
        ''' Logging function for this strategy'''
        dt = dt or self.datas[0].datetime.date(0)
        print('%s, %s' % (dt.isoformat(), txt))

    def __init__(self):
        # Keep a reference to the "close" line in the data[0] dataseries
        self.dataclose = self.datas[0].close
        self.order = None
        self.buyprice = None
        self.buycomm = None

        # Add a MovingAverageSimple indicator
        self.sma = bt.indicators.SimpleMovingAverage(
            self.datas[0], period=self.params.maperiod)

        
    def notify_trade(self, trade):
        if not trade.isclosed:
            return

        self.log('OPERATION PROFIT, GROSS %.2f, NET %.2f' %
                 (trade.pnl, trade.pnlcomm))
    
    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            # Buy/Sell order submitted/accepted to/by broker - Nothing to do
            return

        # Check if an order has been completed
        # Attention: broker could reject order if not enough cash
        if order.status in [order.Completed]:
            if order.isbuy():
                self.log(
                    'BUY EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                    (order.executed.price,
                     order.executed.value,
                     order.executed.comm))

                self.buyprice = order.executed.price
                self.buycomm = order.executed.comm

            elif order.issell():
                self.log('SELL EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                         (order.executed.price,
                          order.executed.value,
                          order.executed.comm))

            self.bar_executed = len(self)

        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('Order Canceled/Margin/Rejected')

        # Write down: no pending order
        self.order = None

    def next(self):
        # Simply log the closing price of the series from the reference
        self.log('Close, %.2f' % self.dataclose[0])
        
        # Check if an order is pending ... if yes, we cannot send a 2nd one
        if self.order:
            return
        
        # Check if we are in the market
        if not self.position:
            if self.dataclose[0] < self.sma:
                    # BUY, BUY, BUY!!! (with all possible default parameters)
                    self.log('BUY CREATE, %.2f' % self.dataclose[0])
                    self.buy()
        else:
            # Already in the market ... we might sell
            if self.dataclose[0] < self.sma[0]:
                # SELL, SELL, SELL!!! (with all possible default parameters)
                self.log('SELL CREATE, %.2f' % self.dataclose[0])

                # Keep track of the created order to avoid a 2nd order
                self.order = self.sell()
        
def get_data(code, start_time='20100101', end_time='20210210'):

    ts.set_token('a339e517ed9b1cb97cda578c2ee8fa829ef50d13ae3623a113227777')
    pro = ts.pro_api()
    
    df  = ts.pro_bar(ts_code=code, adj='qfq', start_date=start_time, end_date=end_time)
    df.index = pd.to_datetime(df.trade_date)
    df.sort_index(ascending=True, inplace=True)
    df['volume'] = df['vol']
    df['openinterest'] = 0
    df = df[['open', 'high', 'low', 'close', 'volume', 'openinterest']]
    return df 


if __name__ == '__main__':
    cerebro = bt.Cerebro()
    cerebro.addstrategy(TestStrategy)
    cerebro.broker.setcash(100000.0)
    cerebro.broker.setcommission(commission=0.001)
    
    data = bt.feeds.PandasData(dataname=get_data('688158.SH'))
    
    cerebro.adddata(data)

    print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())

    cerebro.run()

    print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())

, OPERATION PROFIT, GROSS -4.32, NET -4.49
2020-02-28, Close, 76.92
2020-02-28, BUY CREATE, 76.92
2020-03-02, BUY EXECUTED, Price: 79.42, Cost: 79.42, Comm 0.08
2020-03-02, Close, 79.53
2020-03-02, SELL CREATE, 79.53
2020-03-03, SELL EXECUTED, Price: 81.72, Cost: 79.42, Comm 0.08
2020-03-03, OPERATION PROFIT, GROSS 2.30, NET 2.14
2020-03-03, Close, 95.43
2020-03-04, Close, 86.21
2020-03-05, Close, 83.42
2020-03-05, BUY CREATE, 83.42
2020-03-06, BUY EXECUTED, Price: 80.42, Cost: 80.42, Comm 0.08
2020-03-06, Close, 81.00
2020-03-06, SELL CREATE, 81.00
2020-03-09, SELL EXECUTED, Price: 76.92, Cost: 80.42, Comm 0.08
2020-03-09, OPERATION PROFIT, GROSS -3.50, NET -3.65
2020-03-09, Close, 78.45
2020-03-09, BUY CREATE, 78.45
2020-03-10, BUY EXECUTED, Price: 78.22, Cost: 78.22, Comm 0.08
2020-03-10, Close, 84.43
2020-03-11, Close, 78.22
2020-03-11, SELL CREATE, 78.22
2020-03-12, SELL EXECUTED, Price: 75.93, Cost: 78.22, Comm 0.08
2020-03-12, OPERATION PROFIT, GROSS -2.29, NET -2.44
2020-03-12,